# python延时和超时

### 单线程阻塞延时

In [1]:
import time

def time_sleep(seconds=3):
    start = time.time()
    for i in range(seconds):
        print(i)
        time.sleep(1)
    end = time.time()
    print('run time: {}'.format(end-start))
    
time_sleep()

0
1
2
run time: 3.0014352798461914


### 单线程非阻塞延时

In [3]:
import time

def time_sleep(seconds=3):
    timeout = time.time() + seconds  # 3s delay
    for i in range(seconds + 20):
        print(i)
        time.sleep(1)
        if timeout < time.time(): # compare the timestamps
            break
    print("time out !")


start = time.time()
time_sleep()
end = time.time()
print("run time: {}".format(end - start))


0
1
2
time out !
run time: 3.001466989517212


### 多线程非阻塞延时

这个例子中, 会先执行完 `threading_main`, 5s后, 才会执行 `threading_sub`,

子线程函数可以带参 `threading.Timer(interval, function, args=[], kwargs={})`

In [8]:
import time
import threading

begin_time, end_time = 0, 0

def threading_main():
    global begin_time
    print("main thread start")
    begin_time = time.time()
    thrd = threading.Timer(5.0, threading_sub, args = ["sub thread"])
    thrd.start()
    print("main thread: end")

def threading_sub(name):
    global begin_time, end_time
    print(name + ": hello")
    end_time = time.time()
    print("sub thread run time: {}".format(end_time-begin_time))

start = time.time()
threading_main()
end = time.time()
print("run time: {}".format(end - start))

main thread start
main thread: end
run time: 0.001968860626220703
sub thread: hello
sub thread run time: 5.013208627700806


### 多线程阻塞延时

使用 join 语句, 让主线程等待子线程完成后才继续执行

子线程函数可以带参 `threading.Timer(interval, function, args=[], kwargs={})`

In [9]:
import time
import threading

def threading_main():
    print("main thread: start")
    thrd = threading.Timer(5.0, threading_sub, args = ["sub thread"])
    thrd.start()
    print("main thread: wait")
    thrd.join()     # add this line
    # thrd.join(timeout=2)  # just wait 2s then continue
    print("main thread: end")

def threading_sub(name):
    print(name + ": hello")


start = time.time()
threading_main()
end = time.time()
print("run time: {}".format(end - start))

main thread: start
main thread: wait
sub thread: hello
main thread: end
run time: 5.007183313369751


### 装饰器

装饰器, 使用KThread,.localtrace结束线程. (通用性最好, 性能较低)

In [20]:
import sys
import threading

class Timeout(Exception):
    """function run timeout"""

class KThread(threading.Thread):

    def __init__(self, *args, **kwargs):
        threading.Thread.__init__(self, *args, **kwargs)
        self.killed = False

    def start(self):
        """Start the thread."""
        self.__run_backup = self.run
        # Force the Thread to install our trace.
        self.run = self.__run
        threading.Thread.start(self)

    def __run(self):
        """Hacked run function, which installs the trace."""
        sys.settrace(self.globaltrace)
        self.__run_backup()
        self.run = self.__run_backup

    def globaltrace(self, frame, why, arg):
        if why == 'call':
            return self.localtrace
        else:
            return None

    def localtrace(self, frame, why, arg):
        if self.killed:
            if why == 'line':
                raise SystemExit()
        return self.localtrace

    def kill(self):
        self.killed = True

def timeout(timeout, default=None, try_except=False):
    """Timeout decorator, parameter in timeout."""
    def timeout_decorator(func):
        def new_func(oldfunc, result, oldfunc_args, oldfunc_kwargs):
            result.append(oldfunc(*oldfunc_args, **oldfunc_kwargs))

        """Wrap the original function."""
        def func_wrapper(*args, **kwargs):
            result = []
            # create new args for _new_func, because we want to get the func
            # return val to result list
            new_kwargs = {
                'oldfunc': func,
                'result': result,
                'oldfunc_args': args,
                'oldfunc_kwargs': kwargs
            }

            thd = KThread(target=new_func, args=(), kwargs=new_kwargs)
            thd.start()
            thd.join(timeout)
            # timeout or finished?
            isAlive = thd.isAlive()
            thd.kill()

            if isAlive:
                if try_except is True:
                    raise Timeout("{} Timeout: {} seconds.".format(func, timeout))
                return default
            else:
                return result[0]

        func_wrapper.__name__ = func.__name__
        func_wrapper.__doc__ = func.__doc__
        return func_wrapper

    return timeout_decorator

if __name__ == "__main__":
    import time

    @timeout(5)
    def count(name):
        for i in range(10):
            print("{}: {}".format(name, i))
            time.sleep(1)
        return "finished"

    try:
        print(count("thread1"))
        print(count("thread2"))
    except Timeout as e:
        print(e)

thread1: 0
thread1: 1
thread1: 2
thread1: 3
thread1: 4
None
thread2: 0
thread2: 1
thread2: 2
thread2: 3
thread2: 4
None


将上面的例子, 改为函数调用模式, 这样timeout参数可灵活设置!

In [17]:
import sys
import threading

class Timeout(Exception):
    """function run timeout"""

class KThread(threading.Thread):

    def __init__(self, *args, **kwargs):
        threading.Thread.__init__(self, *args, **kwargs)
        self.killed = False

    def start(self):
        """Start the thread."""
        self.__run_backup = self.run
        # Force the Thread to install our trace.
        self.run = self.__run
        threading.Thread.start(self)

    def __run(self):
        """Hacked run function, which installs the trace."""
        sys.settrace(self.globaltrace)
        self.__run_backup()
        self.run = self.__run_backup

    def globaltrace(self, frame, why, arg):
        if why == 'call':
            return self.localtrace
        else:
            return None

    def localtrace(self, frame, why, arg):
        if self.killed:
            if why == 'line':
                raise SystemExit()
        return self.localtrace

    def kill(self):
        self.killed = True

def timeout_call(timeout, func, args=(), kwargs=None, default=None, try_except=False):
    def new_func(oldfunc, result, oldfunc_args, oldfunc_kwargs):
            result.append(oldfunc(*oldfunc_args, **oldfunc_kwargs))

    result = []
    kwargs = {} if kwargs is None else kwargs
    # create new args for _new_func, because we want to get the func
    # return val to result list
    new_kwargs = {
        'oldfunc': func,
        'result': result,
        'oldfunc_args': args,
        'oldfunc_kwargs': kwargs
    }

    thd = KThread(target=new_func, args=(), kwargs=new_kwargs)
    thd.start()
    thd.join(timeout)
    # timeout or finished?
    isAlive = thd.isAlive()
    thd.kill()

    if isAlive:
        if try_except is True:
            raise Timeout("{} Timeout: {} seconds.".format(func, timeout))
        return default
    else:
        return result[0]

if __name__ == "__main__":
    import time

    def count(name):
        for i in range(10):
            print("{}: {}".format(name, i))
            time.sleep(1)
        return "finished"

    try:
        print(timeout_call(5, count, ["thread1"]))
        print(timeout_call(5, count, ["thread2"]))
    except Timeout as e:
        print(e)

thread1: 0
thread1: 1
thread1: 2
thread1: 3
thread1: 4
None
thread2: 0
thread2: 1
thread2: 2
thread2: 3
thread2: 4
None


缺陷
1. 整体的执行效率会慢一点。因为每次执行一句python语句，都会有一个判断的过程。

2. 因为其本质是使用将函数使用重载的线程来控制，一旦被添加装饰器的函数内部使用了线程或者子进程等复杂的结构，而这些线程和子进程其实是无法获得超时控制的，所以可能导致外层的超时控制无效。

In [27]:
import gevent
from gevent import Timeout

time_to_wait = 5 # seconds

class TooLong(Exception):
    """function run timeout"""

try:
    with Timeout(time_to_wait, TooLong):
        gevent.sleep(10)
except Timeout as e:
    print(e)

TooLong: 

### 常见误区

sleep、wait、join 不能直接用来实现或替代超时功能

尤其是 join(timeout) 方法里的 timeout 很容易让初学者误解，以为调用了  join(n) 就是 n 秒后线程超时结束,
其实 timeout 只是将主线程阻塞，它只告诉join等待子线程运行多久，如果超时后，主线程和子线程还是各自向下继续运行，因此你必须调用 isAlive() 来决定是否超时发生——如果子线程还活着, 表示本次 join() 调用超时了。

In [31]:
from time import sleep, time
import sys, threading
from queue import Queue
from threading import Thread

# reload(sys)
# sys.setdefaultencoding('utf-8')


def processNum(num):
    num_add = num + 1
    sleep(3)
    print(str(threading.current_thread()) + ": " + str(num) + " → " + str(num_add))


class ProcessWorker(Thread):
    def __init__(self, queue):
        Thread.__init__(self)
        self.queue = queue

    def run(self):
        while True:
            num = self.queue.get()
            processNum(num)
            self.queue.task_done()


thread_arr = []


def main():
    ts = time()
    queue = Queue()
    for x in range(10):
        worker = ProcessWorker(queue)
        worker.daemon = True
        worker.start()
        thread_arr.append(worker)
    for num in range(10):
        queue.put(num)
    # queue.join()
    for _ in thread_arr:
        _.join(2)
    print("cost time is: {:.2f}s".format(time() - ts))


if __name__ == "__main__":
    main()

<ProcessWorker(Thread-18, started daemon 12504)>: 9 → 10
<ProcessWorker(Thread-24, started daemon 8384)>: 6 → 7
<ProcessWorker(Thread-16, started daemon 15088)>: 8 → 9
<ProcessWorker(Thread-23, started daemon 2032)>: 2 → 3
<ProcessWorker(Thread-17, started daemon 15192)>: 0 → 1
<ProcessWorker(Thread-21, started daemon 2308)>: 4 → 5
<ProcessWorker(Thread-25, started daemon 6332)>: 7 → 8
<ProcessWorker(Thread-20, started daemon 16588)>: 3 → 4
<ProcessWorker(Thread-22, started daemon 4696)>: 5 → 6
<ProcessWorker(Thread-19, started daemon 16956)>: 1 → 2
cost time is: 20.01s


### func_timeout

* tested func_timeout with python 2.7, 3.4, 3.5, 3.6. It should work on other versions as well.
* Works on windows, linux/unix, cygwin, mac

In [44]:
import time
from func_timeout import func_set_timeout, FunctionTimedOut

start_time, end_time = 0, 0

@func_set_timeout(2.5)
def mytest2():
    global start_time
    start_time = time.time()
    print("Start:", start_time)
    for i in range(1, 10):
        print("%d seconds have passed" % i)
        time.sleep(10)
        
if __name__ == '__main__':
    try:
        mytest2()
    except FunctionTimedOut as e:
        end_time = time.time()
        print('end:', end_time)
        print('mytest2:', e)
        print('run time:', end_time-start_time)

Start: 1534992661.995972
1 seconds have passed
end: 1534992664.5584252
mytest2: Function mytest2 (args=()) (kwargs={}) timed out after 2.500000 seconds.

run time: 2.562453269958496


### stopit

Windows、Linux都可以使用的

In [38]:
import time
import stopit
import traceback

@stopit.threading_timeoutable()
def infinite_loop():
    # As its name says...
    try:
        print("Start")
        for i in range(1, 10):
            print("%d seconds have passed" % i)
            time.sleep(10)
    except Exception as e:
        print('xxxx')
        traceback.print_exc()

if __name__ == '__main__':
    infinite_loop(timeout=1)

Start
1 seconds have passed
xxxx


Traceback (most recent call last):
  File "<ipython-input-38-4b403c68d8c6>", line 12, in infinite_loop
    time.sleep(10)
stopit.utils.TimeoutException


### timeout_decorator

不支持windows,支持linux

In [39]:
import time
import timeout_decorator

@timeout_decorator.timeout(5)
def mytest():
    print("Start")
    for i in range(1,10):
        time.sleep(1)
        print("{} seconds have passed".format(i))

if __name__ == '__main__':
    mytest()

AttributeError: module 'signal' has no attribute 'SIGALRM'

### timeoutcontext

不支持windows,支持linux

In [40]:
import sys
from time import sleep
from timeoutcontext import timeout

if sys.version_info < (3, 3):
    from timeoutcontext._timeout import TimeoutError

try:
    with timeout(1):
        sleep(2)
except TimeoutError:
    print('timeout')

AttributeError: module 'signal' has no attribute 'SIGALRM'